In [1]:
from autofeat import source

dataset = source.from_kaggle(
    name="home-credit-default-risk",
)

In [2]:
where = (
    dataset
    .table("application_train.csv")
    .data
    .select("SK_ID_CURR")
    .collect()
)

target = (
    dataset
    .table("application_train.csv")
    .data
    .select("TARGET")
    .collect()
    .to_series()
)

: 

In [3]:
from autofeat.transform import Aggregate, Cast, Encode, Identity, Require, Select

transform = (
    Require(lambda table: table.name not in {"application_test.csv", "sample_submission.csv"})
    .then(Select(exclude={"TARGET"}))
    .then(Cast())
    .then(Identity(), Encode())
    .then(Identity(), Aggregate())
)

train = dataset.apply(transform)

In [5]:
features = train.features(where=where)

In [6]:
import sklearn.feature_selection
import xgboost

selector = sklearn.feature_selection.SelectFromModel(
    xgboost.XGBRegressor(missing=float("inf")),
    max_features=10,
)

selector.fit(
    X=features.to_numpy(),
    y=target.to_numpy(),
)

selection = [
    features.columns[i]
    for i, selected in enumerate(selector.get_support())
    if selected
]

selection

['EXT_SOURCE_2 from cast(application_train.csv)',
 'EXT_SOURCE_3 from cast(application_train.csv)',
 'REGION_RATING_CLIENT_W_CITY from cast(application_train.csv)',
 'FLAG_DOCUMENT_3 from cast(application_train.csv)',
 'NAME_EDUCATION_TYPE == Higher education from encode(cast(application_train.csv))',
 'CODE_GENDER == F from encode(cast(application_train.csv))',
 'NAME_INCOME_TYPE == Working from encode(cast(application_train.csv))',
 'CODE_GENDER == M from encode(cast(application_train.csv))',
 'std(MONTHS_BALANCE) from group_by(cast(POS_CASH_balance.csv), SK_ID_CURR)',
 'mean(CNT_DRAWINGS_ATM_CURRENT) from group_by(cast(credit_card_balance.csv), SK_ID_CURR)']